![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps%2FServing&file=Serve+TensorFlow+SavedModel+Format+With+BigQuery.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/Serve%20TensorFlow%20SavedModel%20Format%20With%20BigQuery.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FServing%2FServe%2520TensorFlow%2520SavedModel%2520Format%2520With%2520BigQuery.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/Serve%20TensorFlow%20SavedModel%20Format%20With%20BigQuery.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Serving/Serve%20TensorFlow%20SavedModel%20Format%20With%20BigQuery.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Serve TensorFlow SavedModel Format With BigQuery

Serve model predictions inside BigQuery for [TensorFlow SavedModel](https://www.tensorflow.org/guide/saved_model) format.

BigQuery has a vast set of capabilities related to ML known as BigQuery ML (or BQML for short).
- [BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction)
- [BigQuery ML user journey for models](https://cloud.google.com/bigquery/docs/e2e-journey)

> For many workflow examples with BigQuery ML check out the [Framework Workflows/BQML](../../Framework%20Workflows/BQML/readme.md) folder in this repository!

With these capabilities you can train models directly in BigQuery, import model files for serving inside BigQuery, or connect to remote models for use from BigQuery.

This workflow focuses on importing a model, specifically a TensorFlow SavedModel format model as covered in [this documentation page](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-tensorflow). 

There are limits to cover prior to getting started:
- not all BigQuery ML functions will work with imported models
- Models are limited to sizes under 450MB
- Model files must be in GCS, int he SavedModel format, use a GraphDef version of atleast 20, only use core TensorFlow operation (no tf.contrib operations), no RaggedTensors
- see a [full list here](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-tensorflow#limitations)

What this workflow does:
- Prepare the TensorFlow SavedModel files created in this repository by another workflow: [Keras With JAX Overview](../../Framework%20Workflows/Keras/Keras%20With%20JAX%20Overview.ipynb)
- Setup BigQuery Dataset
- Create Models with the imported model files
- Serve predictions with `ML.PREDICT` function

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [2]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [3]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [57]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('numpy', 'numpy'),
    ('tensorflow', 'tensorflow'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [5]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [11]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [12]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [59]:
REGION = 'us-central1'
SERIES = 'mlops-serving'
EXPERIMENT = 'bigquery-tensorflow'

# gcs bucket name
GCS_BUCKET = PROJECT_ID

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

# make this the BigQuery Project / Dataset / Table prefix to store results
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE = SERIES
BQ_REGION = REGION[0:2] # use a multi region

Packages

In [60]:
# import python package
import os

import tensorflow as tf

# BigQuery
from google.cloud import bigquery

# gcs
from google.cloud import storage

Clients

In [61]:
# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

# gcs client
gcs = storage.Client(project = PROJECT_ID)
bucket = gcs.bucket(GCS_BUCKET)

Parameters:

In [16]:
DIR = f"files/{EXPERIMENT}"

Environment:

In [17]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## Model Files For Examples

The TensorFlow SavedModel files used here were created in this repository by another workflow: [Keras With JAX Overview](../../Framework%20Workflows/Keras/Keras%20With%20JAX%20Overview.ipynb).  That workflow does not need to be run because the resulting model files are within this repository.  If this notebook is being used separate from a full clone of the repository then this section will fetch the files from GitHub.

In [22]:
local_dir = '../../Framework Workflows/Keras/files/keras-overview/tensorflow'

In [27]:
if not os.path.exists(local_dir):
    print('Retrieving files...')
    local_dir = DIR
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Embeddings/files/embeddings-api', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Files are now in folder `{local_dir}`')
else:
    print(f'Files Found in folder `{local_dir}`')             

Files Found in folder `../../Framework Workflows/Keras/files/keras-overview/tensorflow`


In [28]:
for root, _, files in os.walk(local_dir):
    for file in files:
        print(os.path.join(root, file))

../../Framework Workflows/Keras/files/keras-overview/tensorflow/embedding_model/fingerprint.pb
../../Framework Workflows/Keras/files/keras-overview/tensorflow/embedding_model/saved_model.pb
../../Framework Workflows/Keras/files/keras-overview/tensorflow/embedding_model/variables/variables.index
../../Framework Workflows/Keras/files/keras-overview/tensorflow/embedding_model/variables/variables.data-00000-of-00001
../../Framework Workflows/Keras/files/keras-overview/tensorflow/stacked_model/fingerprint.pb
../../Framework Workflows/Keras/files/keras-overview/tensorflow/stacked_model/saved_model.pb
../../Framework Workflows/Keras/files/keras-overview/tensorflow/stacked_model/variables/variables.index
../../Framework Workflows/Keras/files/keras-overview/tensorflow/stacked_model/variables/variables.data-00000-of-00001
../../Framework Workflows/Keras/files/keras-overview/tensorflow/final_model/fingerprint.pb
../../Framework Workflows/Keras/files/keras-overview/tensorflow/final_model/saved_mod

In [56]:
!gcloud storage cp -r '{local_dir}' gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/models

Copying file://../../Framework Workflows/Keras/files/keras-overview/tensorflow/embedding_model/fingerprint.pb to gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model/fingerprint.pb
Copying file://../../Framework Workflows/Keras/files/keras-overview/tensorflow/embedding_model/saved_model.pb to gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model/saved_model.pb
Copying file://../../Framework Workflows/Keras/files/keras-overview/tensorflow/embedding_model/variables/variables.index to gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model/variables/variables.index
Copying file://../../Framework Workflows/Keras/files/keras-overview/tensorflow/embedding_model/variables/variables.data-00000-of-00001 to gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model/variables/variables.data-00000-of-00001
Copying file://../../Framework Workflows/Keras/files/keras-overview/tensorflow/s

In [182]:
def get_signature(uri):
    loaded_model = tf.saved_model.load(uri)
    signatures = loaded_model.signatures
    print(f"Available Signatures for: {uri}")
    for signature_name, signature_fn in signatures.items():
        print(f"\nSignature Name: {signature_name}")
        print("\nInputs:")
        if hasattr(signature_fn, 'structured_input_signature') and signature_fn.structured_input_signature:
            for input_name, input_spec in signature_fn.structured_input_signature[1].items():
                print(f"  {input_name}:")
                print(f"    dtype: {input_spec.dtype}")
                print(f"    shape: {input_spec.shape}")
        else:
            print("  No structured input signature available.")
        print("\nOutputs:")
        if hasattr(signature_fn, "structured_outputs") and signature_fn.structured_outputs:
            for output_name, output_spec in signature_fn.structured_outputs.items():
                print(f"  {output_name}:")
                print(f"    dtype: {output_spec.dtype}")
                print(f"    shape: {output_spec.shape}")
        else:
            print("  No structured output signature available.")
            
    def check_xla(signatures):
        try:
            concrete_func = signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
            graph_def = concrete_func.graph.as_graph_def()
            
            def _check_node(node):
                if "XlaCallModule" in node.op:
                    return True
                for attr in node.attr.values():
                    if attr.func:
                        func_name = attr.func.name
                        for func in graph_def.library.function:
                            if func.signature.name == func_name:
                                for func_node in func.node_def:
                                    if _check_node(func_node):
                                        return True
                return False
            
            for node in graph_def.node:
                if _check_node(node):
                    return True
            return False
        except Exception as e:
            print(f'Error checking XLA: {e}')
            return None

    has_xla = check_xla(signatures)
        
    if has_xla:
        print('XLA Detected - Conversion Required for BQML.')
    else:
        print('XLA Not Detected - Model Should Work With BQML')
            
    print('\n\n')
    
    return has_xla

In [183]:
[blob.name for blob in bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/models')]

['mlops-serving/bigquery-tensorflow/models/embedding_model/fingerprint.pb',
 'mlops-serving/bigquery-tensorflow/models/embedding_model/saved_model.pb',
 'mlops-serving/bigquery-tensorflow/models/embedding_model/variables/variables.data-00000-of-00001',
 'mlops-serving/bigquery-tensorflow/models/embedding_model/variables/variables.index',
 'mlops-serving/bigquery-tensorflow/models/final_model/fingerprint.pb',
 'mlops-serving/bigquery-tensorflow/models/final_model/saved_model.pb',
 'mlops-serving/bigquery-tensorflow/models/final_model/variables/variables.data-00000-of-00001',
 'mlops-serving/bigquery-tensorflow/models/final_model/variables/variables.index',
 'mlops-serving/bigquery-tensorflow/models/stacked_model/fingerprint.pb',
 'mlops-serving/bigquery-tensorflow/models/stacked_model/saved_model.pb',
 'mlops-serving/bigquery-tensorflow/models/stacked_model/variables/variables.data-00000-of-00001',
 'mlops-serving/bigquery-tensorflow/models/stacked_model/variables/variables.index']

In [185]:
models = []
for blob in bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/models'):
    if blob.name.endswith('saved_model.pb'):
        uri = f"gs://{bucket.name}/{blob.name.split('/saved_model.pb')[0]}"
        has_xla = get_signature(uri)
        if has_xla:
            print('Model has XLA and will not work with BQML.')
        else:
            models.append(uri)

Available Signatures for: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model

Signature Name: serve

Inputs:
  keras_tensor_113:
    dtype: <dtype: 'float32'>
    shape: (None, 30)

Outputs:
  output_0:
    dtype: <dtype: 'float32'>
    shape: (None, 4)

Signature Name: serving_default

Inputs:
  keras_tensor_113:
    dtype: <dtype: 'float32'>
    shape: (None, 30)

Outputs:
  output_0:
    dtype: <dtype: 'float32'>
    shape: (None, 4)
XLA Detected - Conversion Required for BQML.



Model has XLA and will not work with BQML.


Available Signatures for: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/final_model

Signature Name: serve

Inputs:
  input_layer:
    dtype: <dtype: 'float32'>
    shape: (None, 30)

Outputs:
  normalized_reconstruction:
    dtype: <dtype: 'float32'>
    shape: (None, 30)
  encoded:
    dtype: <dtype: 'float32'>
    shape: (None, 4)
  denormalized_reconstruction:
    dtype: <dtype: 'float32'>
    shape: (None, 30)
  normalized_RMSE:
    dtype: <dtype: 'float32'>
    shape: (None,)
  normalized_MSLE:
    dtype: <dtype: 'float32'>
    shape: (None,)
  denormalized_MAE:
    dtype: <dtype: 'float32'>
    shape: (None,)
  denormalized_MSLE:
    dtype: <dtype: 'float32'>
    shape: (None,)
  normalized_MAE:
    dtype: <dtype: 'float32'>
    shape: (None,)
  normalized_MSE:
    dtype: <dtype: 'float32'>
    shape: (None,)
  denormalized_MSE:
    dtype: <dtype: 'float32'>
    shape: (None,)
  normalized_reconstruction_errors:
    dtype: <dtype: 'float32'>
    shape: (Non

---
## BigQuery Model Import

### Create/Recall Dataset

In [85]:
dataset = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
dataset.location = BQ_REGION
bq_dataset = bq.create_dataset(dataset, exists_ok = True)

### Create/Recall Table With Preparation For ML

Copy the data from the source while adding columns:
- `transaction_id` as a unique identify for the row
    - Use the `GENERATE_UUID()` function
- `splits` column to randomly assign rows to 'TRAIN", "VALIDATE" and "TEST" groups
    - stratified sampling within the levels of `class` by first assigning row numbers within the levels of `class` then using the with a CASE statment to assign the `splits` level.

In [120]:
job = bq.query(f"""
CREATE OR REPLACE TABLE
#CREATE TABLE IF NOT EXISTS 
    `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
WITH
    add_id AS (
        SELECT *,
            GENERATE_UUID() transaction_id,
            ROW_NUMBER() OVER (PARTITION BY class ORDER BY RAND()) as rn
            FROM `{BQ_SOURCE}`
    )
SELECT * EXCEPT(rn),
    CASE 
        WHEN rn <= 0.8 * COUNT(*) OVER (PARTITION BY class) THEN 'TRAIN'
        WHEN rn <= 0.9 * COUNT(*) OVER (PARTITION BY class) THEN 'VALIDATE'
        ELSE 'TEST'
    END AS splits
FROM add_id
""")
job.result()
(job.ended-job.started).total_seconds()

14.537

In [121]:
raw_sample = bq.query(f'SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` LIMIT 5').to_dataframe()
raw_sample

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,56723.0,-0.207067,0.635891,1.468863,-0.701046,-0.105003,-1.086781,0.761288,-0.344025,0.074596,...,-0.064738,0.395483,-0.354941,0.769522,-0.371157,-0.217015,0.0,0,316fd6dc-ee5b-4d04-99bd-2670821fcf35,TEST
1,18098.0,1.165687,0.409860,1.019294,2.044608,-0.154006,0.177514,-0.224623,-0.042588,0.955046,...,0.160420,0.078300,0.119947,0.739299,-0.063692,0.004084,0.0,0,08830e11-2791-4991-8b8e-b857184147b6,TEST
2,126046.0,2.135118,-0.045835,-3.814614,-0.615515,3.225990,2.696986,0.376352,0.435114,-0.307914,...,-0.139735,0.803295,0.719782,0.744174,-0.104081,-0.100148,0.0,0,5c021261-b2a7-4549-8bc1-60dea2645acc,TEST
3,127177.0,-1.476899,0.364446,2.242367,3.191857,1.834470,2.275351,-0.604366,0.867248,-1.501472,...,-0.490477,-1.634986,0.272646,0.484312,0.018321,0.086282,0.0,0,f0893feb-a359-469c-a9a7-0dff1008211b,TEST
4,99389.0,-0.469834,1.381115,2.227870,2.991743,0.625074,0.534667,0.389924,0.085466,-0.041547,...,-0.248184,-0.135584,-0.292904,0.005450,0.141803,0.167097,0.0,0,10c9c140-e3a5-4548-963a-70b3eaf5760b,TEST


### Add A Column For Feature Array

Combine all the features into a single column, an array of floats, for easier inference.

In [122]:
feature_columns = [col for col in raw_sample.columns if col not in ['splits', 'transaction_id', 'Class']]
feature_columns

['Time',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'Amount']

In [124]:
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
SELECT
    t.*, # EXCEPT(features_array),
    ARRAY[
        {', '.join(feature_columns)}
    ] AS features_array
FROM
    `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS t;
"""

job = bq.query(query)
job.result()
(job.ended - job.started).total_seconds()

14.676

### Review the number of records for each level of `Class` for each of the data splits:

In [125]:
bq.query(f"""
SELECT splits, class,
    count(*) as count,
    ROUND(count(*) * 100.0 / SUM(count(*)) OVER (PARTITION BY class), 2) AS percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
GROUP BY splits, class
""").to_dataframe()

,splits,class,count,percentage
0,TEST,1,50,10.16
1,TRAIN,1,393,79.88
2,VALIDATE,1,49,9.96
3,TEST,0,28432,10.00
4,TRAIN,0,227452,80.00
5,VALIDATE,0,28431,10.00


### Create The Models

In [186]:
models

[]

In [127]:
bq_models = []
for model in models:
    bq_model = f"{BQ_PROJECT}.{BQ_DATASET}.{SERIES}-{EXPERIMENT}-{model.split('/')[-1]}"
    job = bq.query(f"""
    CREATE OR REPLACE MODEL `{bq_model}`
        OPTIONS(
            MODEL_TYPE = 'TENSORFLOW',
            MODEL_PATH = '{model}/*'
        )
    """)
    job.result()
    bq_models.append(bq_model)
    print(f"Created BigQuery Model:\n\tName: {bq_model}\n\tGCS URI: {model}\n\tTime (seconds): {(job.ended-job.started).total_seconds()}")

Created BigQuery Model:
	Name: statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-embedding_model
	GCS URI: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model
	Time (seconds): 12.267
Created BigQuery Model:
	Name: statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-final_model
	GCS URI: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/final_model
	Time (seconds): 12.761
Created BigQuery Model:
	Name: statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-stacked_model
	GCS URI: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/stacked_model
	Time (seconds): 17.363


In [128]:
bq_models

['statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-embedding_model',
 'statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-final_model',
 'statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-stacked_model']

### Predictions With ML.PREDICT

In [131]:
bq.query(f"""
SELECT *
FROM ML.PREDICT (MODEL `{bq_models[1]}`,(
    SELECT features_array as input_layer #keras_tensor_113
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
    WHERE splits = 'TEST' and class = 1
    LIMIT 10)
  )
""").to_dataframe()

BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/statmike-mlops-349915/queries/b502b308-28ff-4234-b336-c502563c4ce7?maxResults=0&location=US&prettyPrint=false: Error when running TensorFlow SavedModel: {{function_node __inference_signature_wrapper_stateful_fn_105942}} {{function_node __inference_signature_wrapper_stateful_fn_105942}} Op type not registered 'XlaCallModule' in binary running on sandbox2. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
	 [[{{node StatefulPartitionedCall}}]]
	 [[StatefulPartitionedCall_1]]

Location: US
Job ID: b502b308-28ff-4234-b336-c502563c4ce7
